In [2]:
import qmcpy as qp
import numpy as np
import scipy.stats
#import pandas as pd
import time
#from matplotlib import pyplot
#pyplot.style.use('../qmcpy/qmcpy.mplstyle')
#colors = pyplot.rcParams['axes.prop_cycle'].by_key()['color']
print("ok")

ok


We generated our points in $d$ dimensions and generated $n$ samples such that $\boldsymbol{x}_i \in [0,1]^d$ for $i = 1, 2, 3, ..., n$. Let matrix $\boldsymbol{X}$ be a $n \times d$ matrix such that
$$
\boldsymbol{X} = 
\begin{bmatrix}
\boldsymbol{x}_1\\
\boldsymbol{x}_2\\
\boldsymbol{x}_3\\
...\\
\boldsymbol{x}_n\\
\end{bmatrix}
$$
which can be rewritten as
$$
\boldsymbol{X} = 
\begin{bmatrix}
\boldsymbol{x}_{(1,1)} & \boldsymbol{x}_{(1,2)} & \boldsymbol{x}_{(1,3)} & ... & \boldsymbol{x}_{(1,d)}\\
\boldsymbol{x}_{(2,1)} & \boldsymbol{x}_{(2,2)} & \boldsymbol{x}_{(2,3)} & ... & \boldsymbol{x}_{(2,d)}\\
\boldsymbol{x}_{(3,1)} & \boldsymbol{x}_{(3,2)} & \boldsymbol{x}_{(3,3)} & ... & \boldsymbol{x}_{(3,d)}\\
... & ... & ... & ... & ...\\
\boldsymbol{x}_{(n,1)} & \boldsymbol{x}_{(n,2)} & \boldsymbol{x}_{(n,3)} & ... & \boldsymbol{x}_{(n,d)}\\
\end{bmatrix}
$$

We want to find our discrepancy in star discrepancy we must calculate
$$
[D(\boldsymbol{X}) ]^2= \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy - \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
Note, the kernel for star discrepancy is $K(\boldsymbol{x}_i,\boldsymbol{x}_j) = \prod_{k=1}^d [2 - \max(\boldsymbol{x}_{i_k}, \boldsymbol{x}_{j_k})]$. Also, $\int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{4}{3})^d$ and $\frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \prod_{k=1}^d (\frac{3 - \boldsymbol{x}_{i_k}^2}{2})$. For the sake of program let,
$$
A = \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{4}{3})^d
$$
$$
B = \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \prod_{k=1}^d (\frac{3 - \boldsymbol{x}_{i_k}^2}{2})
$$
$$
C = \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d [2 - \max(\boldsymbol{x}_{i_k}, \boldsymbol{x}_{j_k})]
$$
So, $D(\boldsymbol{X}) = \sqrt{A - B + C}$. We calculate

In [3]:
def star_disc(x):
    n, d = x.shape

    A = (4/3)**d

    b_1 = (3-x**2)/2
    b_2 = b_1.prod(axis=1)
    B = 2*np.mean(b_2)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 2 - np.maximum(c_1, y)
    c_3 = c_2.prod(axis=2)
    c_4 = np.mean(c_3)
    C = np.mean(c_4)

    disc = (A - B + C)**.5
    return disc

We want to find our discrepancy in L2star discrepancy. The equation to find the discrepancy is the same except we choose a different equation for the kernel. Which our kernel is $K(x,y) = \prod_{k=1}^d (1 - \max(x_k,y_k))$. And so to get our discrepancy we must calculate
$$
[D(\boldsymbol{X}) ]^2= \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy - \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
$$
[D(\boldsymbol{X}) ]^2= (1/3)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (\frac{1 - \boldsymbol{x}_{i_k}^2}{2}) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 - \max(\boldsymbol{x}_{i_k}, \boldsymbol{x}_{j_k}))
$$
$$
D(\boldsymbol{X}) = \sqrt{(1/3)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (\frac{1 - \boldsymbol{x}_{i_k}^2}{2}) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 - \max(\boldsymbol{x}_{i_k}, \boldsymbol{x}_{j_k}))}
$$
So now let A, B, and C be calculated as

$$
A = \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{1}{3})^d
$$
$$
B = \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \prod_{k=1}^d (\frac{1 - \boldsymbol{x}_{i_k}^2}{2})
$$
$$
C = \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 - \max(x_k,y_k))
$$
So, $D(\boldsymbol{X}) = \sqrt{A - B + C}$. We calculate

In [4]:
def L2star_disc(x):
    n, d = x.shape

    A = (1/3)**d

    b_1 = (1 - x**2)/2
    b_2 = b_1.prod(axis=1)
    B = 2*np.mean(b_2)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 1 - np.maximum(c_1, y)
    c_3 = c_2.prod(axis=2)
    c_4 = np.mean(c_3)
    C = np.mean(c_4)

    disc = (A - B + C)**.5
    return disc

We want to find our discrepancy in centered discrepancy. We will choose our kernel to be $K(x,y) = \prod_{k=1}^d (1 + .5|\boldsymbol{x}_k - .5| + .5|\boldsymbol{y}_k - .5| - .5 |\boldsymbol{x}_k - \boldsymbol{y}_k|)$. And so to get our discrepancy we must calculate
$$
[D(\boldsymbol{X}) ]^2= \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy - \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
$$
[D(\boldsymbol{X}) ]^2= (13/12)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| - .5 (\boldsymbol{x}_{i_k} -.5)^2) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| + .5|\boldsymbol{x}_{j_k} - .5| - .5 |\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)
$$
$$
D(\boldsymbol{X}) = \sqrt{(13/12)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| - .5 (\boldsymbol{x}_{i_k} -.5)^2) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| + .5|\boldsymbol{x}_{j_k} - .5| - .5 |\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)}
$$
Let A, B, and C be calculated as

$$
A = \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{13}{12})^d
$$
$$
B = \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| - .5 (\boldsymbol{x}_{i_k} -.5)^2)
$$
$$
C = \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| + .5|\boldsymbol{x}_{j_k} - .5| - .5 |\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)
$$
Of course $D(\boldsymbol{X}) = \sqrt{A - B + C}$. So now we calculate,

In [5]:
def center_disc(x):
    n, d = x.shape

    A = (13/12)**d

    b_1 = 1 + (.5*abs(x - .5)) - (.5*((x -.5)**2))
    b_2 = b_1.prod(axis=1)
    B = 2*np.mean(b_2)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 1 + (.5*abs(c_1 - .5)) + (.5*abs(y - .5)) - (.5*abs(c_1 - y))
    c_3 = c_2.prod(axis=2)
    c_4 = np.mean(c_3)
    C = np.mean(c_4)

    disc = (A - B + C)**.5
    return disc

We want to find our discrepancy in symmetric discrepancy. We will choose our kernel to be $K(x,y) = \prod_{k=1}^d (2 - 2|\boldsymbol{x}_k - \boldsymbol{y}_k|)$. And so to get our discrepancy we must calculate
$$
[D(\boldsymbol{X}) ]^2= \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy - \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
$$
[D(\boldsymbol{X}) ]^2= (4/3)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + 2\boldsymbol{x}_{i_k} - 2 \boldsymbol{x}_{i_k}^2) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (2 - 2|\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)
$$
$$
D(\boldsymbol{X}) = \sqrt{(4/3)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + 2\boldsymbol{x}_{i_k} - 2 \boldsymbol{x}_{i_k}^2) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (2 - 2|\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)}
$$
Let A, B, and C be calculated as

$$
A = \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{4}{3})^d
$$
$$
B = \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + 2\boldsymbol{x}_{i_k} - 2 \boldsymbol{x}_{i_k}^2)
$$
$$
C = \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (2 - 2|\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)
$$
Of course $D(\boldsymbol{X}) = \sqrt{A - B + C}$. So now we calculate,

In [6]:
def symmetric_disc(x):
    n, d = x.shape

    A = (4/3)**d

    b_1 = 1 + 2*x - (2*(x**2))
    b_2 = b_1.prod(axis=1)
    B = 2*np.mean(b_2)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 2 - (2*abs(c_1 - y))
    c_3 = c_2.prod(axis=2)
    c_4 = np.mean(c_3)
    C = np.mean(c_4)

    disc = (A - B + C)**.5
    return disc

This and next cell are play cells. So the L2-star discrepancy works just as accurately as the scipy discrepancy; however, confused on the centered discrepancy because in my code I square root to get $D(\boldsymbol{X})$. However in scipy that doesn't seem to be the case. To prove it. I printed out the following outputs. So for the centered discrepancy, this would mean that either I was not suppose to square root or scipy forgot to square root.

In [34]:
#L2-star discrepancy
A = qp.IIDStdUniform(dimension=3)
X = A.gen_samples(n=1024)
def SY(x):
    n, d = x.shape

    A = (4/3)**d

    b_1 = 1 + 2*x - (2*(x**2))
    B = b_1.prod(axis=1)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 2 - (2*abs(c_1 - y))
    C = c_2.prod(axis=2)
    return A, B, C
#print(qp.discrepancy(SY, X))
#print(qp.discrepancy('Sy', X))
print(qp.discrepancy('L2', X))
print(scipy.stats.qmc.discrepancy(X, method='L2-star'))
print("symmetric")
print(symmetric_disc(X))

0.007718971341304397
0.007718971341300802
symmetric
0.06587810944246257


In [10]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
# Stacking 2 1-d arrays
c = np.stack((a, b),axis=0)
print(c)

[[1 2 3]
 [4 5 6]]


In [29]:
A = qp.IIDStdUniform(dimension=3)
X = A.gen_samples(n=4)
#print(np.repeat(3, 4))
#x = np.array([[1,2],[3,4]])
#print(np.repeat(x, 2))
#print(np.repeat(x, 3, axis=1))
#print(np.repeat(x, [1, 2], axis=0))
n, d = X.shape
print(X)
X_expanded = np.repeat(X[np.newaxis, :, :], n, axis= 0)
print(X_expanded)
print(X_expanded[1, 1, 2])
print(X_expanded.shape)
Y_expanded = np.zeros((n,n,d)) + X
print(Y_expanded - X_expanded)

[[0.68102616 0.01422612 0.55449789]
 [0.97211304 0.58789959 0.20441936]
 [0.45921496 0.44525294 0.58294856]
 [0.76007396 0.19434441 0.84834015]]
[[[0.68102616 0.01422612 0.55449789]
  [0.97211304 0.58789959 0.20441936]
  [0.45921496 0.44525294 0.58294856]
  [0.76007396 0.19434441 0.84834015]]

 [[0.68102616 0.01422612 0.55449789]
  [0.97211304 0.58789959 0.20441936]
  [0.45921496 0.44525294 0.58294856]
  [0.76007396 0.19434441 0.84834015]]

 [[0.68102616 0.01422612 0.55449789]
  [0.97211304 0.58789959 0.20441936]
  [0.45921496 0.44525294 0.58294856]
  [0.76007396 0.19434441 0.84834015]]

 [[0.68102616 0.01422612 0.55449789]
  [0.97211304 0.58789959 0.20441936]
  [0.45921496 0.44525294 0.58294856]
  [0.76007396 0.19434441 0.84834015]]]
0.20441936173325836
(4, 4, 3)
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
